In [1]:
'''imports'''

import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
'''entrada da primeira página de resultado de busca e geração de lista de matérias'''

url_input = str(input('Entre a URL da primeira página da busca: '))
qtd_paginas = int(np.ceil(int(input('Quantos resultados existem nessa busca? '))/25))

materias=[] # lista para armazenar as URLs das matérias
re_compile = 'https://www1.folha.uol.com.br/fsp/'
# geração das urls de página de busca:
url_busca = [str(url_input)+'&sr='+str((i*25)+1) for i in range(qtd_paginas)]
    
# laço para varrer as páginas de busca e empilhar as urls na lista materias:
for url in url_busca:
    conteudo_bs = BeautifulSoup(requests.get(url).content, 'html.parser')

    # busca a url das matérias dentro da página de busca (apenas contendo www1..../fsp/):
    for materia in conteudo_bs.findAll('a', attrs={'href': re.compile("^http://www1.folha.uol.com.br/fsp/")}):
        materias.append(materia.get('href'))
        
print('Listagem de matérias realizada! Matérias adicionadas à lista: '+ str(len(materias)))

Entre a URL da primeira página da busca: https://search.folha.uol.com.br/search?q=copa+do+mundo+futebol&periodo=personalizado&sd=12%2F06%2F2014&ed=12%2F07%2F2014&site=jornal
Quantos resultados existem nessa busca? 25
Listagem de matérias realizada! Matérias adicionadas à lista: 25


In [3]:
'''abre matérias da lista, busca, limpa e salva o título em uma lista e o texto em outra'''

titulos=[] # lista dos títulos das matérias
textos=[] # lista contendo o texto das matérias

# laço abrindo matéria por matéria da lista materias:
for mat in materias:
    materia_bs = BeautifulSoup(requests.get(mat).content, 'html.parser')
    
    # busca o título e armazena na lista titulos:
    titulos.append(materia_bs.find('p', 'title').text)
    
    # limpa o texto a partir do título e armazena na lista textos:
    t1 = materia_bs.find('div', id='articleNew').text
    t2 = materia_bs.find('p', 'title').text
    textos.append(BeautifulSoup(t1[t1.index(t2)+len(t2):],'html.parser'))

# correção do título da matéria após 'O tabu' - EXCEÇÃO:
titulos = ['Painel do Leitor' if tit == '' else tit for tit in titulos]
titulos[titulos.index('O tabu')+1] = 'Painel na Copa'   

In [4]:
'''cria os arquivos e armazena neles o título seguido do corpo de texto'''

arquivos=[] # inicia a lista para criação de arquivos e armazenamento dos textos
dir=input(str('Qual o diretório onde serão criados os arquivos? Colocar o endereço com a "\\" no final: '))
size=len(materias)

for i in range(size):
    arquivo_nome='2014_'+str(i+1)+'.txt'
    arquivo=open(dir+arquivo_nome,'w+')
    empilhado=open(dir+'2014_empilhado.txt','a')
    arquivo.write(str(titulos[i])+str(textos[i]))
    empilhado.write(str(titulos[i])+str(textos[i]))
    arquivo.close()
    empilhado.close()
    arquivos.append(arquivo_nome)

Qual o diretório onde serão criados os arquivos? Colocar o endereço com a "\" no final: C:\Users\ruan_\Desktop\Projeto\teste\
